In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime


In [2]:
df = pd.read_csv('wiki_data.csv')
df=df[df.columns[1:]]
nan_values = df[df.columns[:26]].isna().sum().sort_values()
columns_keeping = nan_values[nan_values<6000].index
df = df[columns_keeping]
df['Based on'] = df['Based on'].apply(lambda x: 0 if pd.isna(x) else 1)

C:\Users\jacob\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (36,48,68,69,71,72,75,78,82,100,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,134,135,136,137,138,139,140,141,145,146,147,148,149,150,151,155,158,159,165,166,167,168,169,170,171,172,173,175,176,177,178,179,180,181,182,183,184,185,186,187,188,194,195,196,200,201,202,204,205,206,207,208,209,210,211,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
for col in df.columns[1:]:
    try:
        df[col] = df[col].str[3:-3]
    except:
        print(col)
        

Based on


In [4]:
df['Running time'] = df['Running time'].str.extract(r'(\d+)')
df['Running time'] = pd.to_numeric(df['Running time'])
df['Starring'] = df['Starring'].str.split(',')
df['Produced by'] = df['Produced by'].str.split(',')
df['Release date'] = df['Release date'].str.split('(') 


In [5]:
df['Release date length'] = df['Release date'].apply(lambda x:len(x) if type(x) == list else x)
df['Release date'] =  df['Release date'][df['Release date length']!=10]

In [6]:
 def clean_release(row):
        if type(row)==list:
            if len(row)==1:
                return row[0]
            else:
                return row[1][:-1]
        else:
            row
            
df['Release date']=df['Release date'].apply(clean_release).str.split(')').str[0]

In [7]:
def fix_dates (row):
    if pd.isna(row):
        return row
    else:
        try : 
            return datetime.strptime(row, "%Y-%m-%d")
        except:
            try:
                return datetime.strptime(row, "%B %d, %Y")
            except:
                return row
    

df['Release date'] = df['Release date'].apply(fix_dates)

In [8]:
import re 
df = df[df['Box office'].str.contains('$', na=False, regex=False)]  # remove any non dollar currency
df =df[~df['Box office'].str.contains('admission',na=False,regex=False)]#removes the 5 rows with admission in the box office col

def clean_box(row):
    box = row[1:]
    box =box.replace('$','')
    box = re.split(" |\(|\[", box)
    box = box[0].split('\\')[0]
    
    if bool(re.match(r".*million.*", row)):
        try: return float(box)*1000000
        except: return np.nan
    elif bool(re.match(r".*billion.*", row)):
        try: return float(box)*1000000000
        except: return np.nan  
        
    else: 
        try: return float(box.replace(',',''))
        except: np.nan
    
    

df['Box office'] = df['Box office'].apply(clean_box)

In [9]:
df['Budget'].isna().value_counts()

False    4892
True     1015
Name: Budget, dtype: int64

In [10]:
df['million'] = df['Budget'].str.contains('millio',na=False,regex=False)
df['Budget'] = df['Budget'].str[1:].apply(lambda x: re.split(' |\[', str(x))).str[0].str.replace(',','').str.split('\\').str[0]


In [11]:
def clean_budget(x):
    if '–' in x:
        try: return (float(x.replace(',','').split('–')[0])+float(x.replace(',','').split('–')[1]))/2 
        except: return np.nan
    else:
        try: return float(x)
        except: np.nan
df['Budget'] = df['Budget'].apply(clean_budget)

In [18]:
def million(x):
    budget = x['Budget']
    if x['million'] == True:
        return budget*1000000
    else:
        return budget
df['Budget'] = df.apply(million,axis=1)

In [19]:
df.head()

,Title,Running time,Directed by,Starring,Release date,Produced by,Distributed by,Music by,Edited by,Cinematography,...,Box office,Country,Budget,Written by,Productioncompanies,Screenplay by,Based on,Productioncompany,Release date length,million
0,The Adventures of Ford Fairlane,102.0,Renny Harlin,"[Andrew Dice Clay', 'Wayne Newton', 'Priscil...",1990-07-11 00:00:00,"[Steve Perry', 'Joel Silver]",20th Century Fox,Yello,Michael Tronick,Oliver Wood,...,21400000.0,United States,20000000.0,NaN,NaN,"David Arnott', 'James Cappe', 'Daniel Waters",1,Silver Pictures,2.0,True
1,"After Dark, My Sweet",114.0,James Foley,"[Jason Patric', 'Rachel Ward', 'Bruce Dern',...",1990-05-17 00:00:00,[Ric KidneyRobert Redlin],Avenue Pictures,Maurice Jarre,Howard E. Smith,Mark Plummer,...,2700000.0,United States,6000000.0,NaN,NaN,James FoleyRobert Redlin,1,Avenue Pictures,5.0,True
2,Air America,113.0,Roger Spottiswoode,"[Mel Gibson', 'Robert Downey Jr.', 'Nancy Tr...",1990-08-10 00:00:00,[Daniel MelnickMario Kassar],TriStar Pictures,Charles Gross,John BloomLois Freeman-Fox,Roger Deakins,...,57700000.0,United States,32500000.0,NaN,Carolco PicturesIndieProd Company,"John Eskow', 'Richard Rush",1,NaN,2.0,True
3,Alice,106.0,Woody Allen,"[Alec Baldwin', 'Blythe Danner', 'Judy Davis...",1990-12-25 00:00:00,[Robert Greenhut],Orion Pictures,NaN,Susan E. Morse,Carlo Di Palma,...,7331647.0,United States,12000000.0,Woody Allen,NaN,NaN,0,NaN,2.0,True
4,Almost an Angel,95.0,John Cornell,"[Paul Hogan', 'Elias Koteas', 'Linda Kozlowski]",1990-12-19 00:00:00,[John Cornell],Paramount Pictures,Maurice Jarre,David Stiven,Russell Boyd,...,6939946.0,United States,25000000.0,Paul Hogan,NaN,NaN,0,NaN,1.0,True


In [13]:
df.isna().sum()

Title                      0
Running time              30
Directed by               22
Starring                  49
Release date               2
Produced by               34
Distributed by            38
Music by                 218
Edited by                145
Cinematography           305
Language                 509
Box office                74
Country                 1245
Budget                  1127
Written by              2400
Productioncompanies     2192
Screenplay by           3567
Based on                   0
Productioncompany       4469
Release date length        0
million                    0
dtype: int64

In [14]:
df.shape

(5907, 21)